In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy
import math 
import sklearn.feature_extraction
import sklearn.manifold
import operator
import itertools
import re
import time
import random
import argparse
import os

from collections import Counter
from scipy.stats import pearsonr, spearmanr
from keras import backend as K
from keras.layers import Input, Dense, Flatten, Reshape, Dropout
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.utils import plot_model



Using TensorFlow backend.


In [10]:
df_311 = pd.read_csv("../data/311_service_requests.csv")

/home/ec2-user/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
len(df_311.groupby("complaint_type"))

44

In [13]:
for group in df_311.groupby("complaint_type"):
    print(group)

('Air Quality',          unique_key            created_date             closed_date  \
9911       28471608  07/14/2014 10:27:00 PM  07/22/2014 09:00:00 PM   
12693      28529224  07/22/2014 11:16:00 AM  07/23/2014 08:00:00 AM   
19566      28651877  08/09/2014 02:07:00 PM                     NaN   
21459      28685748  08/14/2014 04:22:00 PM  08/21/2014 05:15:00 PM   
25933      28773031  08/28/2014 09:51:00 AM  09/04/2014 03:45:00 PM   
26126      28777079  08/28/2014 08:18:00 PM  09/03/2014 02:45:00 PM   
29038      37365830  10/06/2017 01:30:00 PM  10/06/2017 06:15:00 PM   
29461      28840317  09/08/2014 06:29:00 AM  09/10/2014 06:00:00 PM   
31655      28882153  09/15/2014 11:19:00 PM  09/23/2014 01:45:00 AM   
32226      28892501  09/16/2014 07:21:00 PM  09/19/2014 03:00:00 PM   
32336      28894910  09/16/2014 07:49:00 PM  09/23/2014 09:15:00 PM   
34985      28946328  09/24/2014 08:04:00 AM  09/30/2014 04:45:00 PM   
35293      28951820  09/24/2014 11:50:00 PM  09/30/2014 11:30

481818      MANHATTAN       NaN        NaN  )
('DORIS Literature Request',         unique_key            created_date closed_date agency_acronym  \
228210    38356192  01/31/2018 01:47:00 PM         NaN            DEP   
309915    38365754  02/01/2018 04:50:00 PM         NaN            DEP   
322052    38367107  02/01/2018 04:45:00 PM         NaN            DEP   
464691    38390529  02/05/2018 10:48:00 AM         NaN            DEP   
468220    38394802  02/05/2018 10:31:00 AM         NaN            DEP   
468641    38393762  02/05/2018 10:36:00 AM         NaN            DEP   
469772    38396878  02/05/2018 10:59:00 AM         NaN            DEP   
481874    38239726  01/18/2018 10:40:00 AM         NaN            DEP   
483370    38295268  01/24/2018 04:57:00 PM         NaN            DEP   
483540    38303054  01/25/2018 01:12:00 PM         NaN            DEP   

                                   agency_name            complaint_type  \
228210  Department of Environmental Protectio

463215 -73.926691  )
('Healthcare Facilities',         unique_key            created_date closed_date agency_acronym  \
301268    34473683  10/05/2016 12:15:44 PM         NaN           FDNY   
434303    36487162  06/19/2017 12:22:30 PM         NaN           FDNY   
539279    32765039  02/24/2016 11:00:03 AM         NaN           FDNY   
540011    32781411  02/26/2016 01:41:28 PM         NaN           FDNY   
624096    34017097  08/05/2016 10:01:34 AM         NaN           FDNY   
786051    34474841  10/05/2016 12:17:11 PM         NaN           FDNY   

                                              agency_name  \
301268  Bureau of Fire Prevention - Hazmat Unit - Insp...   
434303  Bureau of Fire Prevention - Hazmat Unit - Insp...   
539279  Bureau of Fire Prevention - Hazmat Unit - Insp...   
540011  Bureau of Fire Prevention - Hazmat Unit - Insp...   
624096  Bureau of Fire Prevention - Hazmat Unit - Insp...   
786051  Bureau of Fire Prevention - Hazmat Unit - Insp...   

             

('Water Quality',          unique_key            created_date             closed_date  \
7529       28421819  07/07/2014 02:05:00 PM  07/24/2014 11:14:00 AM   
12615      28527576  07/22/2014 10:58:00 PM                     NaN   
20850      28673547  08/13/2014 11:30:00 AM                     NaN   
21134      28679505  08/13/2014 10:35:00 AM                     NaN   
35858      28964465  09/27/2014 06:51:00 AM  09/30/2014 09:08:00 AM   
43141      29111573  10/20/2014 05:50:00 PM                     NaN   
46096      29183197  10/30/2014 09:10:00 PM  10/31/2014 09:15:00 AM   
56395      29488993  12/12/2014 07:34:00 PM  12/16/2014 09:32:00 AM   
57118      29515465  12/16/2014 11:39:00 AM  12/30/2014 12:00:00 AM   
57778      29533830  12/18/2014 10:30:00 AM  12/26/2014 12:00:00 AM   
61119      29629266  01/03/2015 02:45:00 AM  01/05/2015 09:08:00 AM   
62772      29695303  01/11/2015 01:32:00 PM  02/18/2015 12:00:00 AM   
62945      29700415  01/12/2015 02:27:00 PM  01/13/2015 11: